In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.sandbox.google.com/github/https-deeplearning-ai/sc-gc-c4-gemini-public/blob/main/lesson-6/L6_colab_function_calling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Cost Estimate

The estimated cost for running this notebook once using your Google Cloud account should be less than 0.10 USD (as of August 2024). Get the latest Gemini costs [here](https://cloud.google.com/vertex-ai/generative-ai/pricing).


# SETUP

This is follow up to the [How to Set Up your Google Cloud Account](https://learn.deeplearning.ai/courses/large-multimodal-model-prompting-with-gemini/lesson/9/how-to-set-up-your-google-cloud-account-|-try-it-out-yourself-[optional]) instructions from the course, [Large Multimodal Model Prompting with Gemini](https://learn.deeplearning.ai/courses/large-multimodal-model-prompting-with-gemini/lesson/1/introduction) on the [Learning Platform](https://learn.deeplearning.ai) of [DeepLearning.AI](https://www.deeplearning.ai).

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart Runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticate your Notebook Environment (Colab Only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

**NOTE:** The Gmail email address you use to authenticate this lesson colab must be the same as the one you used to set up your Google Cloud account and your Project.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information and Initialize Vertex AI SDK

**Add _your_ Project ID below**, which you created while following the [How to Set Up your Google Cloud Account](https://learn.deeplearning.ai/courses/large-multimodal-model-prompting-with-gemini/lesson/9/how-to-set-up-your-google-cloud-account-|-try-it-out-yourself-[optional]) instructions. If your `Project ID` was `dlai-shortcourse-on-gemini`, then you can run the cell below as it is. Otherwise, be sure to change it.

You can also look up your Project ID in your [Project Dashboard](https://console.cloud.google.com/projectselector2/home/dashboard).

In [2]:
PROJECT_ID = "dlai-shortcourse-on-gemini"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

# IN COURSE VIDEO

Lesson video starts from below.

# [Lesson 6: Integrating Real-Time Data with Function Calling](https://learn.deeplearning.ai/courses/large-multimodal-model-prompting-with-gemini/lesson/7/integrating-real-time-data-with-function-calling)

In this lesson, you'll learn how to integrate real-time data with function calling to get a response from LLMs.

## Asking for Real-Time Information from LLMs

In [3]:
from vertexai.generative_models import GenerativeModel

Please note that Google's Gemini model `gemini-1.5-flash-001` [is being deprecated (soon)](https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations). Therefore, it is necessary to switch to the recommended replacement model, `gemini-2.0-flash-lite`.

- Load the [gemini-2.0-flash-lite](https://ai.google.dev/gemini-api/docs/models#gemini-2.0-flash-lite) model.

In [4]:
model = GenerativeModel("gemini-2.0-flash-lite")

- You are asking for information based on today's date.

**UPDATE**: The newer `gemini-2.0-flash-lite` model *might* actually output today's exchange rate when you run the cells below.

In [5]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)

In [6]:
print(response.text)

Unfortunately, I do not have access to real-time information, including live exchange rates. The exchange rate between euros and dollars fluctuates constantly throughout the day.

To find the current exchange rate, I recommend checking one of the following sources:

*   **A reputable financial website:** such as Google Finance, Yahoo Finance, Bloomberg, or Reuters.
*   **A currency converter website:** Many websites offer currency conversion tools that update in real-time.
*   **Your bank's website or app:** Most banks provide current exchange rates.




## Function Calling with LLMs

- [Frankfurter](https://www.frankfurter.app), an open source API, lets you fetch current and historical foreign exchange rates published by the European Central Bank.

In [7]:
url = "https://api.frankfurter.app/latest"

In [8]:
import requests

- Get the response from Frankfurter for latest/today's exchange rates.
- The base currency is in Euros.

In [9]:
response = requests.get(url)

**Note**: The exchange rates you will see are of the current date you are running this lab, hence they will likely differ from the exchange rates you see in the lecture video.

In [10]:
response.text

'{"amount":1.0,"base":"EUR","date":"2025-04-03","rates":{"AUD":1.7448,"BGN":1.9558,"BRL":6.2411,"CAD":1.5636,"CHF":0.9538,"CNY":8.0919,"CZK":24.972,"DKK":7.4613,"GBP":0.84163,"HKD":8.6311,"HUF":401.58,"IDR":18548,"ILS":4.0919,"INR":94.63,"ISK":144.7,"JPY":162.17,"KRW":1615.11,"MXN":22.223,"MYR":4.9293,"NOK":11.4235,"NZD":1.9047,"PHP":63.295,"PLN":4.196,"RON":4.9774,"SEK":10.7205,"SGD":1.4804,"THB":38.046,"TRY":42.097,"USD":1.1097,"ZAR":20.952}}'

In [11]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

### Declaring Functions

- `FunctionDeclaration` lets you create functions for LLMs.
- In this example of exchange rates, the input parameters for the REST API are:
    - `currency_from` : String : Currency to convert from
    - `currency_to` : String : Currency to convert to
    - `currency_date` : String : Date to fetch the exchange rate for
- The currency codes must be in [ISO 4217](https://www.iban.com/currency-codes) format (for example, "EUR" for Euros)

In [12]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies "
                "between countries",
    parameters={
        "type": "object",
        "properties": {
            "currency_date": {
                "type": "string",
                "description": "A date that must always be in "
                               "YYYY-MM-DD format or the value "
                               "'latest' if a time period is not "
                               "specified"
            },
            "currency_from": {
                "type": "string",
                "description": "The currency to convert from "
                               "in ISO 4217 format"
            },
            "currency_to": {
                "type": "string",
                "description": "The currency to convert to "
                               "in ISO 4217 format"
            }
        },
        "required": [
            "currency_from",
            "currency_date",
        ]
    },
)

### Define a Tool

- `Tool` includes the function declaration that the model uses to generate a response.

In [13]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

- Now that you have declared a Function and define your Tool, let's see how the model responses to your prompt.

**UPDATE:** For the newer `gemini-2.0-flash-lite` model, you have to explicilty mention in the prompt that you are wishing to execute a function call. Oterwise, it will not make the call (i.e. it will not return any arguments).

In [14]:
prompt = """What is the exchange rate from Australian dollars to
Swedish krona? How much is 500 Australian dollars worth in
Swedish krona?
Please use the 'get_exchange_rate' function to retrieve the latest exchange rate and then calculate the converted amount.
It is crucial to use the function to get the most up-to-date information"""

In [15]:
response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [16]:
response.candidates[0].content

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
    }
  }
}

### External API Call

- The model does not make the external API call for you.
- Use `requests` library in Python to make that call for you.
- Turn the model response into a Python Dictionary.

In [17]:
params = {}
for key, value in response.candidates[0].content.parts[0]\
                          .function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

In [18]:
url = f"https://api.frankfurter.app/{params['date']}"

In [19]:
api_response = requests.get(url, params=params)

**Note**: The exchange rates you will see are of the current date you are running this lab, hence they will likely differ from the exchange rates you see in the lecture video.

In [20]:
api_response.text

'{"amount":1.0,"base":"AUD","date":"2025-04-03","rates":{"SEK":6.1443}}'

### Passing the Information Back to the LLM

In [21]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)

**Note**: The exchange rates you will see are of the current date you are running this lab, hence they will likely differ from the exchange rates you see in the lecture video.

In [22]:
response.candidates[0].content.parts[0].text

'The latest exchange rate is 1 Australian dollar = 6.1443 Swedish krona.  Therefore, 500 Australian dollars is worth 500 * 6.1443 = 3072.15 Swedish krona.\n'